An analysis of my Insurance options with ExtensisHR for 2022

The Three plans are detailed in the embedded PDF files at the bottom of the document.

layout:
    create dataframe (medical calendar)
        for each expense type:
            df: date, exepnse desc, expense category
        merge dfs
        sort by date
        add columns for expense, lag, cumsum (for different categories of rolling expenses) 
        deductable_met, out_of_pocket_met
    code up each plan
        premium
        specialty formula
        pcp formula
        prescription formula

In [78]:
from datetime import date
import random

import numpy as np
import pandas as pd

# This is used to embed the PDF files into the notebook. It is not used in the analysis.
from IPython.display import IFrame

In [51]:
# mprint (Multi-print) is a utility function I use to print multiple outputs, on separate lines, in one call.
def mprint(*items, spacing:int=1, sep:str=None) -> None:
    if sep:
        end = '\n' + sep + ('\n' * max(1, spacing - 1)) 
    else:
        end = ('\n' * max(0, spacing))

    for item in items:
        print(item, end=end)

**Defining my Expected Medical Expenses**

I want to create my anticipated medical calendar. This will of course be a guesstimate.



In [199]:
def update_line_premium(line, plan):
    line.cost = plan['premium']
    line.total_cost_running_total = line.previous_total_cost_running_total + plan['premium']
    return line

def update_line_pcp(line, plan):
    return line

def update_line_specialist(line, plan):
    return line

def update_line_test(line, plan):
    return line

def update_line_prescription(line, plan):
    return line

In [191]:
def calc_expense(line, plan):
    if   line.event == 'premium':      line = update_line_premium(line, plan)
    elif line.event == 'pcp':          line = update_line_pcp(line, plan)
    elif line.event == 'specialist':   line = update_line_specialist(line, plan)
    elif line.event == 'test':         line = update_line_test(line, plan)
    elif line.event == 'prescription': line = update_line_prescription(line, plan)
    return line

In [206]:
def create_plan():
    events = {
        'premium': {
            'cost': 28,
            'deductable_applies': False, 
            'out_of_pocket_applies': False, 
        },
        'pcp': {
            # 'deductable': ,
            'coinsurance': 0.1,
            'deductable_applies': True,
            'out_of_pocket_applies': True, 
        },
        'specialist': {
            # 'deductable': ,
            'coinsurance': 0.1,
            'deductable_applies': True,
            'out_of_pocket_applies': True, 
        },  
        'prescription': {
            'deductable': 45,
            # 'coinsurance': ,
            'deductable_applies': True,
            'out_of_pocket_applies': True,
        }, 
        'tests': {
            # 'deductable': ,
            'coinsurance': 0.1,
            'deductable_applies': True,
            'out_of_pocket_applies': True,
        }, 
    }

    plan_b = {
        'name': 'National HSA 3000 F',
        'premium': 28,
        'deductable': 3000,
        'out_of_pocket_max': 5500,
        'events': events
    }
    
    return plan_b

In [276]:
def create_calendar():
        
    start = pd.DataFrame({'date': [pd.Timestamp('2021-12-31')],
                         'event': None, 'detail': None, 'self_pay_cost': 0})
    
    premia = pd.DataFrame({'date': pd.date_range('2022-01-01', periods=24, freq='SM'),
                           'event': 'premium', 'detail': None, 'self_pay_cost': 0})

    random_seed = 42
    visits_pcp_unscheduled_doy = [f'2022{random.randint(1,365)}' for _ in range(1,5)]
    visits_pcp_unscheduled_ts = pd.to_datetime(visits_pcp_unscheduled_doy, format='%Y%j').to_series()
    visits_pcp_scheduled_ts = pd.date_range('2022-01-01', periods=4, freq='QS').to_series()
    visits_pcp_ts = visits_pcp_scheduled_ts.append(visits_pcp_unscheduled_ts)
    pcp = pd.DataFrame({'date': visits_pcp_ts,
                        'event': 'pcp', 'detail': None, 'self_pay_cost': 90})

    therapist = pd.DataFrame({'date': pd.date_range('2022-01-01', periods=52, freq='W').to_series(),
                              'event': 'specialist', 'detail': 'therapist', 'self_pay_cost': 150})

    random_seed = 52
    visits_pt_doy = [f'2022{random.randint(1,365)}' for _ in range(1,16)]
    visits_pt_ts = pd.to_datetime(visits_pt_doy, format='%Y%j').to_series()
    pt = pd.DataFrame({'date': visits_pt_ts,
                       'event': 'specialist', 'detail': 'Physical Therapy', 'self_pay_cost': 225})

    random_seed = 62
    visits_neuro_doy = [f'2022{random.randint(1,365)}' for _ in range(1,7)]
    visits_neuro_ts = pd.to_datetime(visits_pt_doy, format='%Y%j').to_series()
    neuro = pd.DataFrame({'date': visits_neuro_ts,
                          'event': 'specialist', 'detail': 'neurologist', 'self_pay_cost': 250})

    random_seed = 72
    tests_neuro_doy = [f'2022{random.randint(1,365)}' for _ in range(1,4)]
    tests_neuro_ts = pd.to_datetime(visits_pt_doy, format='%Y%j').to_series()
    tests = pd.DataFrame({'date': tests_neuro_ts,
                          'event': 'tests', 'detail': None, 'self_pay_cost': 350})

    scripts_ts = pd.date_range('2022-01-01', periods=12, freq='MS').to_series()
    scripts = pd.DataFrame({'date': pd.date_range('2022-01-01', periods=12, freq='MS').to_series(),
                            'event': 'prescription', 'detail': None, 'self_pay_cost': 115})
    
    calendar = pd.concat([start, premia, pcp, therapist, pt, neuro, tests, scripts]).set_index('date').sort_index()
    return calendar

In [329]:
def cal_get_extended_columns(cal):
    cal['cost'] = 0
    cal['deductable_running_total'] = 0
    cal['out_of_pocket_running_total'] = 0
    cal['total_cost_running_total'] = 0
    cal['self_pay_running_total'] = 0    
    cal['deductable_met'] = False
    cal['out_of_pocket_met'] = False

    return cal

In [351]:
cal = create_calendar()
cal = cal_get_extended_columns(cal)
plan = create_plan()
cal.head(3)

,event,detail,self_pay_cost,cost,cost_type,deductable_applies,deductable_running_total,out_of_pocket_running_total,total_cost_running_total,deductable_met,out_of_pocket_met
date,,,,,,,,,,,
2021-12-31,None,None,0,0,,False,0,0,0,False,False
2022-01-01,prescription,None,115,0,,False,0,0,0,False,False
2022-01-01,pcp,None,90,0,,False,0,0,0,False,False


In [363]:
def update_line_premium(row, plan, rt_deduct, rt_oopocket, rt_total):
    row['cost_type'] = 'premium'
    row['cost'] = plan['premium']
    row['total_cost_running_total'] = rt_total + plan['premium']
    row['deductable_running_total'] = rt_deduct
    row['out_of_pocket_running_total'] = rt_oopocket
    row['total_cost_running_total'] = rt_total + row['cost']
    return row

def update_line_service(row, plan, rt_deduct, rt_oopocket, rt_total, deduct_met, oopocket_met):
    type = row.event
    details = plan['events'][type]
    expense_type, expense_num = next((t, details[t]) \
                                     for t in details.keys() \
                                     if t in ['coinsurance', 'deductable'])
    row.cost_type = expense_type
    
    if expense_type == 'deductable':
        if details['deductable_applies']:
            row.deductable_applies = True
            print('mark0: is_deduct       ', row.name, row.event, plan['deductable'], expense_num + rt_deduct)
            if deduct_met:
                print('mark1 - deduct_met_prev', row.name, row.event, plan['deductable'], expense_num + rt_deduct)
                row.cost = 0
            elif ((expense_num + rt_deduct) > plan['deductable']):
                print('mark2 - deduct_met_now ', row.name, row.event, plan['deductable'], expense_num + rt_deduct)
                row.cost = (expense_num + rt_deduct) - plan['deductable']
            else:
                row.cost = expense_num
        else:
            if oopocket_met:
                row.cost = 0
            elif ((expense_num + rt_oopocket) > plan['out_of_pocket']):
                row.cost = (expense_num + rt_oopocket) - plan['out_of_pocket']
            else:
                row.cost = expense_num
                
    elif expense_type == 'coinsurance':
        row.deductable_applies = True
        coinsurance_cost = row.self_pay_cost * expense_num
        if row.out_of_pocket_met:
            row.cost = 0
        elif row.deductable_met:
            if ((coinsurance_cost + rt_oopocket) > plan['out_of_pocket_max']):
                row.cost = (coinsurance_cost + rt_oopocket) - plan['out_of_pocket_max']
                # row.out_of_pocket_met = True  
            else:
                row.cost = coinsurance_cost
        else:
            row.cost = row.self_pay_cost
            
    row.deductable_running_total = rt_deduct + row.cost
    # print('mark: 2', row.name, row.event, plan['deductable'], row.deductable_running_total)
    if row.deductable_running_total >= plan['deductable']: row.deductable_met = True
    
    row.out_of_pocket_running_total = rt_oopocket + row.cost
    if row.out_of_pocket_running_total >= plan['out_of_pocket_max']: row.out_of_pocket_met = True
    
    row.total_cost_running_total = rt_total + row.cost
    
    
    return row

def update_running_totals(cal, idx, counts_toward_limits):
    print(cal.iloc[idx-1]['deductable_running_total'] + (cal.iloc[idx]['cost'] * counts_toward_limits))
    cal.iloc[idx]['deductable_running_total'] = cal.iloc[idx-1]['deductable_running_total'] + (cal.iloc[idx]['cost'] * counts_toward_limits)
    cal.iloc[idx]['out_of_pocket_running_total'] = cal.iloc[idx-1]['out_of_pocket_running_total'] + (cal.iloc[idx]['cost'] * counts_toward_limits)
    cal.iloc[idx]['total_cost_running_total'] = cal.iloc[idx-1]['total_cost_running_total'] + cal.iloc[idx]['cost']
    return cal



for idx in range(1,51):
    deduct_met = cal.iloc[idx-1]['deductable_met']
    oopocket_met = cal.iloc[idx-1]['out_of_pocket_met']
    rt_deduct = cal.iloc[idx-1]['deductable_running_total']
    rt_oopocket = cal.iloc[idx-1]['out_of_pocket_running_total']
    rt_total = cal.iloc[idx-1]['total_cost_running_total']

    if cal.iloc[idx]['event'] == 'premium': 
        cal.iloc[idx] = update_line_premium(cal.iloc[idx], plan, rt_deduct, rt_oopocket, rt_total)
                                            
    else:
        cal.iloc[idx] = update_line_service(cal.iloc[idx], plan, rt_deduct, rt_oopocket, rt_total, 
                                            deduct_met, oopocket_met,)
        
    # # cal = update_running_totals(cal,idx, counts_toward_limits)
    # print(cal.iloc[idx-1]['deductable_running_total'] + (cal.iloc[idx]['cost'] * counts_toward_limits))
    # cal.iloc[idx]['deductable_running_total'] = cal.iloc[idx-1]['deductable_running_total'] + (cal.iloc[idx]['cost'] * counts_toward_limits)
    # cal.iloc[idx]['out_of_pocket_running_total'] = cal.iloc[idx-1]['out_of_pocket_running_total'] + (cal.iloc[idx]['cost'] * counts_toward_limits)
    # cal.iloc[idx]['total_cost_running_total'] = cal.iloc[idx-1]['total_cost_running_total'] + cal.iloc[idx]['cost']

    
cal.head(50)

mark0: is_deduct        2022-01-01 00:00:00 prescription 3000 45.0
mark0: is_deduct        2022-02-01 00:00:00 prescription 3000 930.0
mark0: is_deduct        2022-03-01 00:00:00 prescription 3000 2400.0
mark0: is_deduct        2022-04-01 00:00:00 prescription 3000 3084.0
mark1 - deduct_met_prev 2022-04-01 00:00:00 prescription 3000 3084.0
mark0: is_deduct        2022-05-01 00:00:00 prescription 3000 4614.0
mark2 - deduct_met_now  2022-05-01 00:00:00 prescription 3000 4614.0


/Users/jason.thomas/dev/jupyter/venv/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


,event,detail,self_pay_cost,cost,cost_type,deductable_applies,deductable_running_total,out_of_pocket_running_total,total_cost_running_total,deductable_met,out_of_pocket_met
date,,,,,,,,,,,
2021-12-31,None,None,0,0.0,,False,0.0,0.0,0.0,False,False
2022-01-01,prescription,None,115,45.0,deductable,True,45.0,45.0,45.0,False,False
2022-01-01,pcp,None,90,90.0,coinsurance,True,135.0,135.0,135.0,False,False
2022-01-02,specialist,therapist,150,150.0,coinsurance,True,285.0,285.0,285.0,False,False
2022-01-09,specialist,therapist,150,150.0,coinsurance,True,435.0,435.0,435.0,False,False
2022-01-15,premium,None,0,28.0,premium,False,435.0,435.0,463.0,False,False
2022-01-16,specialist,therapist,150,150.0,coinsurance,True,585.0,585.0,613.0,False,False
2022-01-23,specialist,therapist,150,150.0,coinsurance,True,735.0,735.0,763.0,False,False
2022-01-30,specialist,therapist,150,150.0,coinsurance,True,885.0,885.0,913.0,False,False


In [209]:
def deductable_applies(line, plan):
    return plan['events'][line.event]['deductable_applies']
def out_of_pocket_applies(line, plan):
    return plan['events'][line.event]['out_of_pocket_applies']

c['deductable_applies'] = c.apply(deductable_applies, axis='columns', plan=plan)
c['out_of_pocket_applies'] = c.apply(out_of_pocket_applies, axis='columns', plan=plan)

c

,event,detail,deductable_applies,out_of_pocket_applies
date,,,,
2022-01-01,prescription,None,True,True
2022-01-01,pcp,None,True,True
2022-01-02,specialist,therapist,True,True
2022-01-09,specialist,therapist,True,True
2022-01-15,premium,None,False,False
...,...,...,...,...
2022-12-15,premium,None,False,False
2022-12-17,pcp,None,True,True
2022-12-18,specialist,therapist,True,True


In [201]:
calendar = calendar.apply(calc_expense, axis='columns', plan=plan)
calendar

,event,detail,cost,cost_type,deducatable_applies,deductable_running_total,deductable_met,out_of_pocket_met,out_of_pocket_running_total,total_cost_running_total,previous_deductable_running_total,previous_out_of_pocket_running_total,previous_total_cost_running_total
date,,,,,,,,,,,,,
2022-01-01,prescription,None,0,,False,0,False,False,0,0.0,NaN,NaN,NaN
2022-01-01,pcp,None,0,,False,0,False,False,0,0.0,0.0,0.0,0.0
2022-01-02,specialist,therapist,0,,False,0,False,False,0,0.0,0.0,0.0,0.0
2022-01-09,specialist,therapist,0,,False,0,False,False,0,0.0,0.0,0.0,0.0
2022-01-15,premium,None,28,,False,0,False,False,0,28.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-11,specialist,therapist,0,,False,0,False,False,0,0.0,0.0,0.0,0.0
2022-12-15,premium,None,28,,False,0,False,False,0,28.0,0.0,0.0,0.0
2022-12-18,specialist,therapist,0,,False,0,False,False,0,0.0,0.0,0.0,0.0


In [293]:
x = False
10 * x

0

In [80]:
# attribution: https://stackoverflow.com/questions/34258892/converting-year-and-day-of-year-into-datetime-index-in-pandas

def compose_date(years, months=1, days=1, weeks=None, hours=None, minutes=None,
                 seconds=None, milliseconds=None, microseconds=None, nanoseconds=None):
    years = np.asarray(years) - 1970
    months = np.asarray(months) - 1
    days = np.asarray(days) - 1
    types = ('<M8[Y]', '<m8[M]', '<m8[D]', '<m8[W]', '<m8[h]',
             '<m8[m]', '<m8[s]', '<m8[ms]', '<m8[us]', '<m8[ns]')
    vals = (years, months, days, weeks, hours, minutes, seconds,
            milliseconds, microseconds, nanoseconds)
    return sum(np.asarray(v, dtype=t) for t, v in zip(types, vals)
               if v is not None)

df = pd.DataFrame({'doy': [49, 65, 81, 97, 113, 129, 145, 161],
                   'year': [2000, 2000, 2000, 2001, 2001, 2001, 2001, 2001]})

df.index = compose_date(df['year'], days=df['doy'])
print(df)

            doy  year
2000-02-18   49  2000
2000-03-05   65  2000
2000-03-21   81  2000
2001-04-07   97  2001
2001-04-23  113  2001
2001-05-09  129  2001
2001-05-25  145  2001
2001-06-10  161  2001


In [3]:
IFrame('National POS 25-6350 F Nov 2021.pdf', width=700, height=350)

In [165]:
expenses_plan_a = {
    'pcp': {
        'deductable': 25,
        # 'coinsurance': ,
        'deductable_applies': False,
    },
    'specialist': {
        # 'deductable': ,
        'coinsurance': 0,
        'deductable_applies': True,
    },  
    'prescription': {
        'deductable': 45,
        # 'coinsurance':0,
        'deductable_applies': False,
    },  
    'tests': {
        # 'deductable': ,
        'coinsurance': 0,
        'deductable_applies': True,
    }, 
}

plan_a = {
    'name': 'National POS 25-6350 F',
    'premium': 0,
    'deductable': 6350,
    'out_of_pocket_max': 6850,
    'expenses': expenses_plan_a
}

In [4]:
IFrame('National HSA 3000 F Nov 2021 .pdf', width=700, height=350)

In [166]:
expenses_plan_b = {
    'pcp': {
        # 'deductable': ,
        'coinsurance': 0.1,
        'deductable_applies': True,
    },
    'specialist': {
        # 'deductable': ,
        'coinsurance': 0.1,
        'deductable_applies': True,
    },  
    'prescription': {
        'deductable': 45,
        # 'coinsurance': ,
        'deductable_applies': True,
    }, 
    'tests': {
        # 'deductable': ,
        'coinsurance': 0.1,
        'deductable_applies': True,
    }, 
}

plan_b = {
    'name': 'National HSA 3000 F',
    'premium': 28,
    'deductable': 3000,
    'out_of_pocket_max': 5500,
    'expenses': expenses_plan_b
}

In [5]:
IFrame('National POS 25-750 F Nov 2021.pdf', width=700, height=350)

In [167]:
expenses_plan_c = {
    'pcp': {
        'deductable': 25,
        # 'coinsurance': ,
        'deductable_applies': False,
    },
     'specialist': {
        'deductable': 50,
        # 'coinsurance': ,
        'deductable_applies': True,
    },  
     'prescription': {
        'deductable': 45,
        # 'coinsurance': ,
        'deductable_applies': False,
    }, 
    'tests': {
        # 'deductable': ,
        'coinsurance': 0.1,
        'deductable_applies': True,
    }, 
}

plan_c = {
    'name': 'National POS 25-750 F',
    'premium': 147.5,
    'deductable': 750,
    'out_of_pocket_max': 3000,
    'expenses': expenses_plan_c
}